# 🧠 Elite Explainable AI (XAI) System: Beyond the Black Box

## 1. Problem Definition & Advanced Context
**Objective**: Predict credit risk with high precision and provide **actionable** explanations.

**Production-Ready Features**:
- 🤖 **Automated Tuning**: Optuna for optimization.
- 💾 **Serialization**: Saving the full pipeline for deployment.
- 🌍 **Counterfactuals**: Actionable user feedback.
- 📊 **Dashboard**: Stakeholder-ready ExplainerDashboard.

In [ ]:
# 🛠️ 1. Setup & Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import lime.lime_tabular
import xgboost as xgb
import dice_ml
import optuna
import joblib  # For serialization
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, 
                             roc_auc_score, confusion_matrix)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.datasets import fetch_openml

# Configuration
import warnings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
shap.initjs()

## 2. Data Loading & Preprocessing

In [ ]:
# Load Data
data = fetch_openml(data_id=31, as_frame=True, parser='auto')
df = data.frame

# Map Target: 'bad' -> 1 (Risk), 'good' -> 0 (Safe)
target_col = 'class'
df['target'] = df[target_col].map({'bad': 1, 'good': 0})
df = df.drop(columns=[target_col])

# Split Features and Target
X = df.drop(columns=['target'])
y = df['target']

# Identify columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Data loaded and split successfully.")

## 3. Advanced Training: Hyperparameter Tuning with Optuna
Instead of guessing parameters, we use Bayesian Optimization to find the best XGBoost configuration.

In [ ]:
def objective(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42
    }
    
    # Create Pipeline with suggested params
    model = xgb.XGBClassifier(**param)
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    
    clf.fit(X_train, y_train)
    
    preds = clf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)
    
    return auc

print("Starting Optuna Optimization...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  # Low trials for demo speed, increase for production

print("Best Hyperparameters:", study.best_params)

In [ ]:
# Train Final Model with Best Params
best_params = study.best_params
best_xgb = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')

final_pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                                 ('classifier', best_xgb)])

final_pipeline.fit(X_train, y_train)

y_pred = final_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

## 4. Model Serialization (Production Ready)
We save the entire pipeline (including preprocessing) so it can be deployed to a server.

In [ ]:
# Save the pipeline
model_filename = 'model_pipeline.joblib'
joblib.dump(final_pipeline, model_filename)
print(f"Model saved to {model_filename}")

# Verification: Try reloading it
loaded_model = joblib.load(model_filename)
print("Model reloaded successfully. Ready for inference.")

## 5. Counterfactual Explanations (DiCE)
Feature importance tells us *what* matters. Counterfactuals tell us *what to do*.
**Scenario**: A customer is rejected. They ask: *"What do I need to change to be approved?"*

In [ ]:
# DiCE requires a slightly different data structure
# We need to pass the raw (untransformed) data to DiCE, but the model expects transformed data.
# So we wrap our pipeline in a class or use DiCE's sklearn backend.

import dice_ml

# Create DiCE Data object
# We use the training data to calculate statistics
train_df = X_train.copy()
train_df['target'] = y_train

d = dice_ml.Data(dataframe=train_df, continuous_features=num_cols, outcome_name='target')

# Create DiCE Model object
m = dice_ml.Model(model=final_pipeline, backend="sklearn", model_type='classifier')

# Initialize DiCE Explainer
exp_dice = dice_ml.Dice(d, m, method="random")

# Generate counterfactuals for a rejected customer
# Pick a customer with prediction = 1 (Bad Risk)
risk_indices = np.where(y_test == 1)[0]
query_idx = risk_indices[0]
query_instance = X_test.iloc[[query_idx]]

print("Query Instance (High Risk):")
print(query_instance)

# Generate CFs
dice_exp = exp_dice.generate_counterfactuals(query_instance, total_CFs=3, desired_class=0)

# Visualize
dice_exp.visualize_as_dataframe(show_only_changes=True)

## 6. Professional Dashboard (ExplainerDashboard)
We can launch a full comprehensive dashboard directly within the notebook or on a separate port. This aggregates SHAP, permutation importance, confusion matrix, and individual predictions.

In [ ]:
# ExplainerDashboard requires a fit model and data
# Note: It handles pipelines, but sometimes it's safer to pass the final estimator and transformed data
# However, the library supports pipelines well now.

print("Building ExplainerDashboard... (This may take a moment)")
explainer = ClassifierExplainer(final_pipeline, X_test, y_test)

# Start an inline dashboard
# You can also run .run(port=8050) to open in a new tab
ExplainerDashboard(explainer, mode='inline').run()

## 7. Production Inference Simulation
This section simulates what happens when the `deployment_test.py` script runs.

In [ ]:
def production_predict(input_data):
    print("Receiving API Request...")
    df_input = pd.DataFrame([input_data])
    
    # Pipeline handles all preprocessing (Scaling, Encoding)
    prob = loaded_model.predict_proba(df_input)[0][1]
    print(f"Risk Probability: {prob:.4f}")
    
    return "REJECT" if prob > 0.5 else "APPROVE"

# Mock API Call
mock_data = X_test.iloc[0].to_dict()
decision = production_predict(mock_data)
print(f"Final Decision: {decision}")